In [1]:
#---Importing packages

import json
import pandas as pd
import numpy as np

In [2]:
#---Reading restaurants data

with open('/Users/tete/628data/bus_pizza.json', 'r') as f:
    bus_train = f.readlines()
    for i in range(len(bus_train)):
        bus_train[i] = json.loads(bus_train[i])
f.close

<function TextIOWrapper.close()>

In [3]:
#---Accounting number of kinds of attributes

attribute = set()
for i in range(len(bus_train)):
    if bus_train[i]['attributes']:
        attribute = set(list(bus_train[i]['attributes'])) | attribute
print(len(attribute))

37


In [4]:
#---Transforming all attributes

for i in range(len(bus_train)):
    for key in attribute:
        bus_train[i][key] = 'NA'
        if key in list(bus_train[i]['attributes']):
            bus_train[i][key] = bus_train[i]['attributes'][key]
            if bus_train[i][key] == 'None':
                bus_train[i][key] = 'NA'

for i in range(len(bus_train)):
    bus_train[i].pop('attributes')

In [5]:
#---Transforming BusinessParking
    
for i in range(len(bus_train)):
    if bus_train[i]['BusinessParking'] != 'NA':
        bus_train[i]['BusinessParking'] = bus_train[i]['BusinessParking'].replace('\'', '\"')
        bus_train[i]['BusinessParking'] = bus_train[i]['BusinessParking'].replace('False', '\"F\"')
        bus_train[i]['BusinessParking'] = bus_train[i]['BusinessParking'].replace('True', '\"T\"') 
        bus_train[i]['BusinessParking'] = json.loads(bus_train[i]['BusinessParking'])
        
for i in range(len(bus_train)):
    for key in ['garage', 'street', 'validated', 'lot', 'valet']:
        bus_train[i]['Parking_'+key] = 'NA'
        if bus_train[i]['BusinessParking'] != 'NA':
            if key in list(bus_train[i]['BusinessParking']):
                bus_train[i]['Parking_'+key] = bus_train[i]['BusinessParking'][key]
                    
for i in range(len(bus_train)):
    bus_train[i].pop('BusinessParking')

In [6]:
#---Transforming Ambience

for i in range(len(bus_train)):
    if bus_train[i]['Ambience'] != 'NA':
        bus_train[i]['Ambience'] = bus_train[i]['Ambience'].replace('\'', '\"')
        bus_train[i]['Ambience'] = bus_train[i]['Ambience'].replace('False', '\"F\"')
        bus_train[i]['Ambience'] = bus_train[i]['Ambience'].replace('True', '\"T\"') 
        bus_train[i]['Ambience'] = json.loads(bus_train[i]['Ambience'])
        
for i in range(len(bus_train)):
    for key in ['romantic', 'intimate', 'touristy', 'hipster', 'divey', 'classy', 'trendy', 'upscale', 'casual']:
        bus_train[i]['Ambience_'+key] = 'NA'
        if bus_train[i]['Ambience'] != 'NA':
            if key in list(bus_train[i]['Ambience']):
                bus_train[i]['Ambience_'+key] = bus_train[i]['Ambience'][key]
                    
for i in range(len(bus_train)):
    bus_train[i].pop('Ambience')

In [7]:
#---Transforming GoodForMeal
    
for i in range(len(bus_train)):
    if bus_train[i]['GoodForMeal'] != 'NA':
        bus_train[i]['GoodForMeal'] = bus_train[i]['GoodForMeal'].replace('\'', '\"')
        bus_train[i]['GoodForMeal'] = bus_train[i]['GoodForMeal'].replace('False', '\"F\"')
        bus_train[i]['GoodForMeal'] = bus_train[i]['GoodForMeal'].replace('True', '\"T\"') 
        bus_train[i]['GoodForMeal'] = json.loads(bus_train[i]['GoodForMeal'])
        
for i in range(len(bus_train)):
    for key in ['dessert', 'latenight', 'lunch', 'dinner', 'brunch', 'breakfast']:
        bus_train[i]['GoodForMeal_'+key] = 'NA'
        if bus_train[i]['GoodForMeal'] != 'NA':
            if key in list(bus_train[i]['GoodForMeal']):
                bus_train[i]['GoodForMeal_'+key] = bus_train[i]['GoodForMeal'][key]
                    
for i in range(len(bus_train)):
    bus_train[i].pop('GoodForMeal')

In [8]:
#---Testing whether their are other variables to be transformed

for i in range(len(bus_train)):
    for key in attribute:
        if key not in ['BusinessParking', 'Ambience', 'GoodForMeal']:
            if bus_train[i][key][0] == '\{':
                print(key)

In [9]:
#---Reading reviews data

with open('/Users/tete/628data/review_train.json', 'r') as f:
    review_train = f.readlines()
    for i in range(len(review_train)):
        review_train[i] = json.loads(review_train[i])
f.close

<function TextIOWrapper.close()>

In [10]:
#---Picking restaurants' reviews

df_bus_train = pd.DataFrame(bus_train)
id = np.array(df_bus_train['business_id'])

review_json = []
for i in range(len(review_train)):
    if review_train[i]['business_id'] in id:
        review_json.append(json.dumps(review_train[i]))
for i in range(len(review_json)):
    open('/Users/tete/628data/review_pizza.json', 'a').write(review_json[i])
    open('/Users/tete/628data/review_pizza.json', 'a').write('\n')

In [11]:
#---Re-reading restaurants' reviews to release the memory

del review_train
del review_json
with open('/Users/tete/628data/review_pizza.json', 'r') as f:
    review_train = f.readlines()
    for i in range(len(review_train)):
        review_train[i] = json.loads(review_train[i])
f.close

<function TextIOWrapper.close()>

In [12]:
#---Calculating average ratings

ratings = {}
account = {}
for x in review_train:
    if x['business_id'] not in list(ratings):
        ratings[x['business_id']] = x['stars']
        account[x['business_id']] = 1
    else:
        ratings[x['business_id']] = ratings[x['business_id']] + x['stars']
        account[x['business_id']] = account[x['business_id']] + 1

ave_ratings = {}
for key in list(ratings):
    ave_ratings[key] = ratings[key]/account[key]

In [13]:
#---Merging average ratings and business data

sorted_ratings = []
for i in range(df_bus_train.shape[0]):
        sorted_ratings.append(ave_ratings[df_bus_train['business_id'][i]])

df_bus_train['ratings'] = sorted_ratings
df_bus_train.to_csv('/Users/tete/628data/bus_pizza.csv', sep = ',', header = True)

In [14]:
#---Generating CSV of review data

df_review_train = pd.DataFrame(review_train)
df_review_train.to_csv('/Users/tete/628data/review_pizza.csv', sep = ',', header = True)